<a href="https://colab.research.google.com/github/1kaiser/PlanetaryBoundaryLayerProcessor/blob/main/PBLH_processor%F0%9F%92%BB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/1kaiser/PlanetaryBoundaryLayerProcessor.git

Cloning into 'PlanetaryBoundaryLayerProcessor'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 13 (delta 4), reused 13 (delta 4), pack-reused 0
Unpacking objects: 100% (13/13), done.


In [2]:
%cd /content/PlanetaryBoundaryLayerProcessor

/content/PlanetaryBoundaryLayerProcessor


# Process to download

In [ ]:
!npm install -g npm
!npm install puppeteer turndown bluebird
!apt-get update
!apt install chromium-chromedriver

/tools/node/bin/npm -> /tools/node/lib/node_modules/npm/bin/npm-cli.js
/tools/node/bin/npx -> /tools/node/lib/node_modules/npm/bin/npx-cli.js
+ npm@8.6.0
added 58 packages from 18 contributors, removed 292 packages and updated 142 packages in 8.133s

added 61 packages, and audited 62 packages in 15s

8 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:7 http://archive.ubun

In [ ]:
%%writefile index.js

'use strict';

const puppeteer = require('puppeteer');
const fs = require('fs');
var TurndownService = require('turndown');
const bluebird = require("bluebird");


(async() => {
  const browser = await puppeteer.launch({executablePath:"/usr/lib/chromium-browser/chromium-browser", args:['--no-sandbox']});
  //console.log(golinks)
    // console.log(golinks[0]["Link"], golinks[0]["Title"])
    let file = '\out.txt'
    if (!fs.existsSync(file)) {
        fs.writeFileSync(file, '', function (err) {
            if (err)
                console.log(err);
        });
    }
    var darray = fs.readFileSync('2021DD.txt', 'utf8').split('\n');
    let file1 = "MD"
    if (!fs.existsSync(file1)) {
        fs.mkdirSync(file1, function (err) {
            if (err)
                console.log(err);
        });
    }
    let withPage = (browser) => async (fn) => {
        const page = await browser.newPage();
        try {
            return await fn(page);
        } catch (err) {
            console.error(err.message);
            await page.close();
            return false;
        } finally {
            await page.close();
        }
    }
    const urls = [];
    const titles = [];
    for (let index = 0; index < darray.length; index++) {
        urls.push(darray[index].split('\,')[4]);
        titles.push(darray[index].split('\,')[5].replace('\r',''));
    }
    var darray = fs.readFileSync('\out.txt', 'utf8').split('\n');
    let count = 0
    bluebird.map(urls, async (url) => {
        if (!darray.join().includes(url)) {
            return withPage(browser)(async (page) => {
                await page._client.send("Page.setDownloadBehavior", {
                    behavior: "allow",
                    downloadPath: file1,
                });
                let completed = []
                let data = false;
                let attempts = 0;
                let viewSource
                while (data == false && attempts < 2) {
                    viewSource = await page.goto(url, {
                        waitUntil: "networkidle0",
                    });
                    data = await page.content();
                    attempts += 1;
                    if (data != false) {
                        fs.writeFile(`MD/${titles[urls.indexOf(url)]}`, await viewSource.buffer(), function(err) {
                            if(err) {
                                return console.log(err);
                            }
                        
                            console.log("The file was saved!");
                        });
                        completed.push(url.concat('\n'))
                        fs.appendFileSync('out.txt', completed.toString())
                        // console.log(titles[urls.indexOf(url)])
                    }
                }
                console.log(urls.length - darray.length - count++, " <<remaining of ", urls.length - darray.length, "with overall links", darray.length + urls.length)
            }).then((r) => ({
                result: r
            }), (e) => ({
                error: e
            }));
        }
    }, {
        concurrency: 7 //<< increase to go parallel tabs using bluebird library
    });

})()


In [ ]:
!node index.js

#zipping

In [ ]:
!zip -r /content/20220313.zip /content/Positive/MD /content/Positive/out.txt